[![Open In Colab](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/badge/open-in-colab.svg)](https://colab.research.google.com/github/crunchdao/quickstarters/blob/master/competitions/broad-obesity-1/quickstarters/baseline/baseline.ipynb)

![Banner](https://raw.githubusercontent.com/crunchdao/quickstarters/refs/heads/feat/obesity/competitions/broad-obesity-1/assets/banner.webp)

# Obesity ML Competition: Tackling metabolic diseases

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

# Setup

The first steps to get started are:
1. Get the setup command
2. Execute it in the cell below

### >> https://hub.crunchdao.com/competitions/broad-obesity-1/submit/notebook

![Reveal token](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/reveal-token.gif)

In [ ]:
# Install the Crunch CLI
%pip install --upgrade crunch-cli

# Setup your local environment
!crunch setup --notebook datacrunch hello --token aaaabbbbccccddddeeeeffff

# Your model

## Setup

In [ ]:
# Install required dependencies
%pip install anndata scanpy

In [ ]:
import os
import typing

# Import your dependencies
import anndata
import numpy as np
import pandas as pd
import scanpy
import scipy
from tqdm.notebook import tqdm

In [ ]:
# Staging URLs
%env API_BASE_URL=http://api.hub.crunchdao.io
%env WEB_BASE_URL=http://hub.crunchdao.io

In [ ]:
import crunch

# Load the Crunch Toolings
crunch_tools = crunch.load_notebook()

## Constants

Store your global values here.

In [ ]:
# @crunch/keep:on

control_label = "NC"

## Understanding the Data

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

The data was downloaded when you setup your local environment and is now available in the `data/` directory.

In [ ]:
def load_data(
    data_directory_path: str = "data",
):
    adata_train = scanpy.read_h5ad(os.path.join(data_directory_path, "obesity_challenge_1.h5ad"))

    return adata_train

In [ ]:
adata_train = load_data()

### Understanding `adata_train`

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

In [ ]:
adata_train.obs

In [ ]:
def to_array(
    adata: anndata.AnnData,
) -> np.typing.NDArray[np.float64]:
    if isinstance(adata.X, np.ndarray):
        return adata.X

    return adata.X.toarray()

In [ ]:
X_train_values = to_array(adata_train)

X_train_values.shape

In [ ]:
def get_centroids(
    adata_train: anndata.AnnData,
    X_train_values: np.typing.NDArray[np.float64],
):
    # Getting the control centroid in the train set
    control_mask = adata_train.obs["gene"] == control_label
    assert control_mask.sum() != 0, "No control cell"
    control_centroid = X_train_values[control_mask].mean(axis=0)

    # Getting the perturbed centroid
    perturbed_mask = ~control_mask
    perturbed_centroid = X_train_values[perturbed_mask].mean(axis=0)

    return control_mask, control_centroid, perturbed_mask, perturbed_centroid

In [ ]:
(
    control_mask,
    control_centroid,
    perturbed_mask,
    perturbed_centroid,
) = get_centroids(
    adata_train,
    X_train_values,
)

control_centroid.shape, perturbed_centroid.shape

In [ ]:
num_validation_sample = perturbed_centroid.shape[0]
prediction = np.ones((num_validation_sample, control_centroid.shape[0])) * control_centroid

prediction.shape

In [ ]:
with open(os.path.join("data", "predict_perturbations.txt"), "r") as fd:
    predict_perturbations = [
        file.strip()
        for file in fd.readlines()
    ]

In [ ]:
adata_train_control = adata_train[control_mask]
pred_proportion_df = pd.DataFrame({"gene": predict_perturbations})
pred_proportion_df["adipo"] = float(adata_train_control.obs["adipo"].mean())
pred_proportion_df["pre_adipo"] = float(adata_train_control.obs["pre_adipo"].mean())
pred_proportion_df["other"] = float(adata_train_control.obs["other"].mean())
pred_proportion_df["lipo"] = float(adata_train_control.obs["lipo"].mean())

pred_proportion_df

## Strategy Implementation

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

### The `train()` Function

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

In [ ]:
def train(
    data_directory_path: str,
    model_directory_path: str,
):
    pass

### The `infer()` Function

Lorem ipsum dolor sit amet, consectetur adipiscing elit.

In [ ]:
def infer(
    data_directory_path: str,
    prediction_directory_path: str,
    model_directory_path: str,
    predict_perturbations: typing.List[str],
):
    print("Loading data...")
    global adata_train, X_train_values
    if "adata_train" not in globals():
        # Optimization for Colab, avoid loading the data twice
        adata_train = load_data(data_directory_path)
        X_train_values = to_array(adata_train)

    print("Extracting centroids...")
    (
        control_mask,
        control_centroid,
        perturbed_mask,
        perturbed_centroid,
    ) = get_centroids(
        adata_train,
        X_train_values,
    )

    print("Infering the prediction...")
    num_validation_sample = perturbed_centroid.shape[0]
    prediction_array = np.ones((num_validation_sample, control_centroid.shape[0])) * control_centroid

    prediction = anndata.AnnData(
        X=prediction_array,
        # obs=adata_train.obs["gene"].to_frame() # TODO fix
    )

    print("Saving the prediction...")
    prediction.write(os.path.join(prediction_directory_path, "prediction.h5ad"))

    print("Infering the proportion...")
    adata_train_control = adata_train[control_mask]
    pred_proportion_df = pd.DataFrame({"gene": predict_perturbations})
    pred_proportion_df["adipo"] = float(adata_train_control.obs["adipo"].mean())
    pred_proportion_df["pre_adipo"] = float(adata_train_control.obs["pre_adipo"].mean())
    pred_proportion_df["other"] = float(adata_train_control.obs["other"].mean())
    pred_proportion_df["lipo"] = float(adata_train_control.obs["lipo"].mean())

    print("Saving the proportion...")
    pred_proportion_df.to_csv(os.path.join(prediction_directory_path, "predict_program_proportion.csv"), index=False)


## Local testing

To make sure your `train()` and `infer()` function are working properly, you can call the `crunch.test()` function that will reproduce the cloud environment locally. <br />
Even if it is not perfect, it should give you a quick idea if your model is working properly.

In [ ]:
crunch_tools.test()

## Results

Once the local tester is done, you can preview the result stored in `prediction/prediction.parquet`.

In [ ]:
prediction = scanpy.read_h5ad(os.path.join("prediction", "prediction.h5ad"))
prediction_X = to_array(prediction)

prediction

In [ ]:
predicted_proportion = pd.read_csv(os.path.join("prediction", "predict_program_proportion.csv"))

predicted_proportion

### Local scoring

You can call the function that the system uses to estimate your score locally.

In [ ]:
gtruth = scanpy.read_h5ad(os.path.join("data", "obesity_challenge_1_local_gtruth.h5ad"))
gtruth_X = to_array(gtruth)

gtruth

In [ ]:
gtruth_proportion = pd.read_csv(os.path.join("data", "program_proportion_local_gtruth.csv"))

#### Transcriptome-wide metrics

These metrics will be computed using top **5000 highly variable genes (HVG)** genes for each perturbation, listed in the file [hvg_gene.csv](./data/hvg_gene.csv). The HVG genes are a subset of all genes measured in the training dataset.

#### Pearson Delta

The Pearson Delta ($\rho$) is the perturbation effects relative to perturbed mean ($\hat{X}_P$) between predicted ($\hat{X}$) and observed ($X$):

$
\displaystyle
\rho(X, \hat{X}) = \frac{1}{|P|} \sum_{p \in P} \text{cor}\left(\hat{X}_p - \hat{X}_{PM'}, X_p - X_{PM}\right)
$

where:
- $X_{PM}$ is the Perturbed Mean defined as the mean gene expression of all the perturbed cells (those receiving a single gene perturbation) in the training dataset,
- $P$ is the set of all perturbation targets,
- $|P|$ is the size of the set $P$,
- $\text{cor}(a, b)$ is the correlation between vectors $a$ and $b$.

---

*Higher is better!*

In [ ]:
def compute_metric_pearson(
    gtruth_X: np.typing.NDArray[np.float64],
    pred_X: np.typing.NDArray[np.float64],
    perturbed_centroid: np.typing.NDArray[np.float64],
) -> float:
    gtruth_X_target = gtruth_X.mean(axis=0)
    pred_X_target = pred_X.mean(axis=0)

    return scipy.stats.pearsonr(
        gtruth_X_target - perturbed_centroid,
        pred_X_target - perturbed_centroid,
    ).statistic


pearson = compute_metric_pearson(
    gtruth_X,
    prediction_X,
    gtruth.uns["perturbed_centroid_train"],
)

print(f"Pearson score: {pearson:.4f}")

#### Maximum Mean Discrepancy (MMD)

The MMD is between the predicted and the observed distributions of single-cell profiles.

Let $X_p^i$ be the true expression profile and $\hat{X}_p^i$ be the predicted expression profile for cell $i$ with perturbation $p$.
  Then we calculate the MMD distance for a particular perturbation using the following formulae:

$
\displaystyle
MMD^2(X_p, \hat{X}_p) =
\frac{1}{N^2} \sum_{i,j} k(X_p^i, X_p^j) +
\frac{1}{N^2} \sum_{k,l} k(\hat{X}_p^k, \hat{X}_p^l) -
\frac{2}{N^2} \sum_{m,n} k(X_p^m, \hat{X}_p^n)
$

where:
- $k(a, b)$ is the Gaussian kernel with the bandwidth from the following list: $[375, 750, 1500, 3000, 6000]$.

---

Then we average the MMD score across all the perturbations using the following formulae:

$
\displaystyle
MMD^2(X, \hat{X}) = \frac{1}{|P|} \sum_{p \in P} MMD^2(X_p, \hat{X}_p)
$

where:
- $|P|$ is the size of the set $P$.

---

*Lower is better!*

In [ ]:
def _gaussian_kernel(source, target, kernel_mul, kernel_num, fix_sigma):
    # Getting the L2 distance
    n_samples = int(source.shape[0]) + int(target.shape[0])
    total = np.concatenate([source, target], axis=0)

    total0 = np.broadcast_to(np.expand_dims(total, axis=0), (int(total.shape[0]), int(total.shape[0]), int(total.shape[1])))
    total1 = np.broadcast_to(np.expand_dims(total, axis=1), (int(total.shape[0]), int(total.shape[0]), int(total.shape[1])))
    L2_distance = np.sum((total0 - total1)**2, axis=2)

    # Now we are ready to scale this using multiple bandwidth
    if fix_sigma:
        bandwidth = fix_sigma
    else:
        bandwidth = np.sum(L2_distance) / (n_samples**2 - n_samples)

    # Now we will create the multiple bandwidth list
    bandwidth /= kernel_mul ** (kernel_num // 2)
    bandwidth_list = [bandwidth * (kernel_mul**i) for i in range(kernel_num)]
    kernel_val = [np.exp(-L2_distance / bandwidth_temp) for bandwidth_temp in bandwidth_list]

    return sum(kernel_val)


def _compute_mmd_batch(X_batch, Y_batch, kernel_mul, kernel_num, fix_sigma, kernel_func):
    num_batch_element = X_batch.shape[0]

    kernels = kernel_func(X_batch, Y_batch, kernel_mul, kernel_num, fix_sigma)
    XX = kernels[:num_batch_element, :num_batch_element]
    YY = kernels[num_batch_element:, num_batch_element:]
    XY = kernels[:num_batch_element, num_batch_element:]
    YX = kernels[num_batch_element:, :num_batch_element]
    mmd_val = np.sum(XX + YY - XY - YX)

    return mmd_val, num_batch_element ** 2


def compute_metric_mmd(
    gtruth_X: np.typing.NDArray[np.float64],
    pred_X: np.typing.NDArray[np.float64],
) -> float:
    kernel_mul = 2.0
    kernel_num = 5
    fix_sigma = None

    num_source = gtruth_X.shape[0]
    num_batches = 30 * 10  # (30 for 20k samples)
    batch_size = num_source // num_batches

    results = [
        _compute_mmd_batch(
            gtruth_X[bidx * batch_size:(bidx + 1) * batch_size, :],
            pred_X[bidx * batch_size:(bidx + 1) * batch_size, :],
            kernel_mul,
            kernel_num,
            fix_sigma,
            _gaussian_kernel
        )
        for bidx in tqdm(range(num_batches))
    ]

    mmd_dist_sum = sum(r[0] for r in results)
    num_element = sum(r[1] for r in results)

    mmd_dist = mmd_dist_sum / num_element
    return mmd_dist


mmd = compute_metric_mmd(
    gtruth_X,
    prediction_X,
)

print(f"MMD score: {mmd:.4f}")

#### Program-level metrics.

These metrics evaluate whether models capture meaningful biological outcomes.

#### L1-distance

There are four cell state proportions for each perturbation, i.e., pre-adipogenic, adipogenic, lipogenic, and other.
For each perturbation $p$, we have the ground truth cell-state proportion $S_p = [preadipo, adipo, lipo, other]$.

Let $S_p^R$ be the vector that has the proportion of the cell states $[preadipo, adipo, other]$. <br />
Let $S_p^L$ denote the condition probability of a cell being in the lipogenic state given the adipogenic state, i.e., $S_p^L = lipo/adipo$.

Then we define the program level loss as:

$
\displaystyle
L1(\hat{S}, S) = \frac{1}{|P|} \sum_{p \in P} 0.75 * | S_p^R - \hat{S}_p^R |_1 + 0.25 * | \hat{S}_p^L - S_p^L |
$

where $|.|_1$ is the L₁-distance, $|P|$ is the number of perturbations, and $\hat{S}_p$ is the predicted cell-state proportions.

---

*Lower is better!*

In [ ]:
def compute_metric_l1_distance(
    true_state_proportion_df: pd.DataFrame,
    pred_state_proprotion_df: pd.DataFrame,
) -> float:
    # Going over all the genes that were perturbed in this set
    unique_perturb_genes = list(true_state_proportion_df["gene"].unique())

    all_l1_loss_list = []
    for gene in unique_perturb_genes:
        # Slicing the column with this gene
        true_gene_df = true_state_proportion_df[true_state_proportion_df["gene"] == gene]
        pred_gene_df = pred_state_proprotion_df[pred_state_proprotion_df["gene"] == gene]

        # print(gene, pred_gene_df.shape[0])
        assert true_gene_df.shape[0] == 1 and pred_gene_df.shape[0] == 1, f"Invalid prediction count for state gene={gene} count={pred_gene_df.shape[0]}!=1"

        # Getting the L1 loss for main  pre, adipo and other
        l1_three = (
            np.abs(true_gene_df.iloc[0]["pre_adipo"] - pred_gene_df.iloc[0]["pre_adipo"]) +
            np.abs(true_gene_df.iloc[0]["adipo"] - pred_gene_df.iloc[0]["adipo"]) +
            np.abs(true_gene_df.iloc[0]["other"] - pred_gene_df.iloc[0]["other"])
        )

        # Getting the L1 loss for lipo by adipo
        numerical_stab_term = 1e-20
        pred_lipo_adipo = pred_gene_df.iloc[0]["lipo"] / (pred_gene_df.iloc[0]["adipo"] + numerical_stab_term)
        true_lipo_adipo = true_gene_df.iloc[0]["lipo"] / (true_gene_df.iloc[0]["adipo"] + numerical_stab_term)
        l1_lipo_adipo = np.abs(true_lipo_adipo - pred_lipo_adipo)

        # Getting the average error
        average_l1 = 0.75 * l1_three + 0.25 * l1_lipo_adipo
        all_l1_loss_list.append(average_l1)

    # Getting the overall average over all the gene perturbation
    l1_loss = np.mean(all_l1_loss_list)
    return float(l1_loss)


l1_distance = compute_metric_l1_distance(
    gtruth_proportion,
    predicted_proportion,
)

print(f"L1-distance score: {l1_distance:.4f}")

## Writing the report

The final step is to write the method description as specified [in the documentation](https://docs.crunchdao.com/competitions/competitions/broad-obesity/crunch-1#file-method-description.md).

You must:
1. Explain how your method works. (5-10 sentences)
2. Describe the reasoning behind your gene panel design. (5-10 sentences)
3. Specify the datasets and any other resources utilized. (5-10 sentences)

The limit is about one page.
<br />
<br />
<br />

---

<span style="font-size: 48px">👇👇👇</span> (double-click the markdown cell below)

---
file: Method description.md
---

<!-- Don't forget to change me -->

# Method Description

Explain how your method works. (5-10 sentences)

# Rationale

Describe the reasoning behind your gene panel design. (5-10 sentences)

# Data and Resources Used

Specify the datasets and any other resources utilized. (5-10 sentences)

<span style="font-size: 48px">👆👆👆</span>

---
<br />
<br />
<br />

# Submit your Notebook

To submit your work, you must:
1. Download your Notebook from Colab
2. Upload it to the platform
3. Create a run to validate it

### >> https://hub.crunchdao.com/competitions/broad-obesity-1/submit/notebook

![Download and Submit Notebook](https://raw.githubusercontent.com/crunchdao/competitions/refs/heads/master/documentation/animations/download-and-submit-notebook.gif)